In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, 
and then re-execute this cell.


In [3]:
#Pre-requisite: If the code is executed in Google Colab, download the dataset and upload it to your google drive in the same location
#Dataset: https://drive.google.com/file/d/19kPdA5v7gtAMU5QbIxHxjWj_HdiYPcZf/view?usp=sharing
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
## Implementation of Baseline GAN 
#1. Importing required packages
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten
from keras.layers import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np

#2. Constructing MusGAN class
class MusGAN():
  def __init__(self):
    self.img_rows = 640
    self.img_cols = 128
    self.channels = 1
    self.img_shape = (self.img_rows, self.img_cols, self.channels)
    self.latent_dim = 64
    self.discriminator_lr = 0.0002
    self.generator_lr = 0.001
    self.discriminator = self.build_discriminator()
    self.discriminator.compile(loss='binary_crossentropy',
                                optimizer=Adam(self.discriminator_lr, 0.5),
                                metrics=['accuracy'])
    self.generator = self.build_generator()
    z = Input(shape=(self.latent_dim,))
    gen_img = self.generator(z)
    self.discriminator.trainable = False
    validity = self.discriminator(gen_img)
    self.combined = Model(z, validity)
    self.combined.compile(loss='binary_crossentropy', optimizer=Adam(self.generator_lr, 0.5))

#3. Defining generator model
  def build_generator(self):
    model = Sequential()
    model.add(Dense(256, input_dim=self.latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(2048))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(4096))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(8192))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(np.prod(self.img_shape), activation='tanh'))
    model.add(Reshape(self.img_shape))
    noise = Input(shape=(self.latent_dim,))
    img = model(noise)
    print("Generator Summary")
    model.summary()
    return Model(noise, img)

#4. Defining discriminator model
  def build_discriminator(self):
    model = Sequential()
    model.add(Flatten(input_shape=self.img_shape))
    model.add(Dense(8192))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(4096))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(2048))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1, activation='sigmoid'))
    img = Input(shape=self.img_shape)
    validity = model(img)
    print("Discrimnator Summary")
    model.summary()
    return Model(img, validity)

#5. Defining the training parameters
  def train(self, epochs, batch_size=32, sample_interval=5000):
    train_data = np.load('/content/drive/My Drive/DL/FMA/shuffled_train.npz')
    self.X_train = train_data['arr_0']
    self.X_train = self.X_train / 127.5 - 1.
    self.X_train = np.expand_dims(self.X_train, axis=3)
    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))
    for epoch in range(epochs):
      idx = np.random.randint(0, self.X_train.shape[0], batch_size)
      real_seqs = self.X_train[idx]
      noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
      g_loss = self.combined.train_on_batch(noise, valid)
      gen_seqs = self.generator.predict(noise)
      d_loss_real = self.discriminator.train_on_batch(real_seqs, valid)
      d_loss_fake = self.discriminator.train_on_batch(gen_seqs, fake)
      d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
      print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
      if epoch % sample_interval == 0:
        self.sample_images(epoch)

#6. Method to generate new image at given sample interval
  def sample_images(self, epoch):
        predictions = self.generator.predict(noise)
        print(predictions.shape)
        print("Reshaping Output data")
        generated_seq = np.reshape(predictions[0], (640,128))
        print("Transforming Output data")
        generated_seq = np.rot90(generated_seq,3)
        print(generated_seq.shape)
        self.convert_to_audio(generated_seq, epoch)
        librosa.display.specshow(generated_seq, y_axis='mel', fmax=8000, x_axis='time')
        plt.colorbar(format='%+2.0f dB')
        plt.savefig("/content/drive/My Drive/DL/GAN/Results/GAN_MUSIC/%d.png" % epoch)
        plt.show()
        plt.close()

In [0]:
#7. Begin training
gan = MusGAN()    
gan.train(epochs=10, batch_size=32, sample_interval=2)

Discrimnator Summary
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 81920)             0         
_________________________________________________________________
dense_15 (Dense)             (None, 8192)              671096832 
_________________________________________________________________
leaky_re_lu_13 (LeakyReLU)   (None, 8192)              0         
_________________________________________________________________
dense_16 (Dense)             (None, 4096)              33558528  
_________________________________________________________________
leaky_re_lu_14 (LeakyReLU)   (None, 4096)              0         
_________________________________________________________________
dense_17 (Dense)             (None, 2048)              8390656   
_________________________________________________________________
leaky_re_lu_15 (LeakyReLU)   (Non